In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 8
tree_depth = 10
batch_size = 512
device = 'cuda'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [7]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.200286865234375 | KNN Loss: 5.570528030395508 | CLS Loss: 1.629758596420288
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 4.334754467010498 | KNN Loss: 2.962836265563965 | CLS Loss: 1.3719182014465332
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 3.275571346282959 | KNN Loss: 2.6373131275177 | CLS Loss: 0.6382583379745483
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 3.2182888984680176 | KNN Loss: 2.552614212036133 | CLS Loss: 0.6656746864318848
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 3.1036977767944336 | KNN Loss: 2.5522866249084473 | CLS Loss: 0.5514110922813416
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 3.072950839996338 | KNN Loss: 2.5594332218170166 | CLS Loss: 0.5135177373886108
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 3.1318154335021973 | KNN Loss: 2.584388017654419 | CLS Loss: 0.5474274754524231
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 3.086531162261963 | KNN Loss: 2.600828170776367 | CLS 

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 2.577674388885498 | KNN Loss: 2.4594333171844482 | CLS Loss: 0.11824096739292145
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 2.6667587757110596 | KNN Loss: 2.52266788482666 | CLS Loss: 0.14409081637859344
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 2.6705310344696045 | KNN Loss: 2.476156711578369 | CLS Loss: 0.19437432289123535
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 2.605355739593506 | KNN Loss: 2.477708578109741 | CLS Loss: 0.127647265791893
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 2.6377131938934326 | KNN Loss: 2.461115837097168 | CLS Loss: 0.176597461104393
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 2.645249605178833 | KNN Loss: 2.4669461250305176 | CLS Loss: 0.17830350995063782
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 2.6023247241973877 | KNN Loss: 2.5080676078796387 | CLS Loss: 0.09425705671310425
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 2.5906553268432617 | KNN Loss: 2.4685647

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 2.548036575317383 | KNN Loss: 2.425438404083252 | CLS Loss: 0.12259820103645325
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 2.602889060974121 | KNN Loss: 2.4497108459472656 | CLS Loss: 0.1531781554222107
Epoch: 007, Loss: 2.5477, Train: 0.9756, Valid: 0.9728, Best: 0.9728
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 2.501044273376465 | KNN Loss: 2.4234397411346436 | CLS Loss: 0.07760453969240189
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 2.5922679901123047 | KNN Loss: 2.467078447341919 | CLS Loss: 0.1251896321773529
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 2.538774251937866 | KNN Loss: 2.4373648166656494 | CLS Loss: 0.10140953212976456
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 2.5473005771636963 | KNN Loss: 2.436347007751465 | CLS Loss: 0.11095353960990906
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 2.5847597122192383 | KNN Loss: 2.475551128387451 | CLS Loss: 0.10920850187540054
Epoch 8 / 200 | iterati

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 2.505141258239746 | KNN Loss: 2.4568467140197754 | CLS Loss: 0.04829449579119682
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 2.4942829608917236 | KNN Loss: 2.391362190246582 | CLS Loss: 0.10292084515094757
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 2.4991743564605713 | KNN Loss: 2.4225046634674072 | CLS Loss: 0.07666979730129242
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 2.461980104446411 | KNN Loss: 2.391669750213623 | CLS Loss: 0.07031026482582092
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 2.5431132316589355 | KNN Loss: 2.470979928970337 | CLS Loss: 0.07213333249092102
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 2.5163300037384033 | KNN Loss: 2.440521001815796 | CLS Loss: 0.07580895721912384
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 2.4715356826782227 | KNN Loss: 2.4231200218200684 | CLS Loss: 0.04841567203402519
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 2.4814271926879883 | KNN Loss

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 2.5127854347229004 | KNN Loss: 2.414885997772217 | CLS Loss: 0.09789931774139404
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 2.5002973079681396 | KNN Loss: 2.405195951461792 | CLS Loss: 0.09510133415460587
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 2.460742473602295 | KNN Loss: 2.411587953567505 | CLS Loss: 0.04915445297956467
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 2.466646671295166 | KNN Loss: 2.4057438373565674 | CLS Loss: 0.06090276688337326
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 2.462177276611328 | KNN Loss: 2.3895647525787354 | CLS Loss: 0.07261242717504501
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 2.5027318000793457 | KNN Loss: 2.4199397563934326 | CLS Loss: 0.0827919989824295
Epoch: 014, Loss: 2.4868, Train: 0.9833, Valid: 0.9797, Best: 0.9797
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 2.4999172687530518 | KNN Loss: 2.444162130355835 | CLS Loss: 0.05575508251786232
Epoch 15 /

...

Epoch 185 / 200 | iteration 170 / 171 | Total Loss: 2.418367862701416 | KNN Loss: 2.410569667816162 | CLS Loss: 0.007798283360898495
Epoch: 185, Loss: 2.4164, Train: 0.9969, Valid: 0.9858, Best: 0.9879
Epoch 186 / 200 | iteration 0 / 171 | Total Loss: 2.4439210891723633 | KNN Loss: 2.420003890991211 | CLS Loss: 0.023917116224765778
Epoch 186 / 200 | iteration 10 / 171 | Total Loss: 2.424248456954956 | KNN Loss: 2.4124906063079834 | CLS Loss: 0.011757914908230305
Epoch 186 / 200 | iteration 20 / 171 | Total Loss: 2.4127585887908936 | KNN Loss: 2.4028172492980957 | CLS Loss: 0.00994129665195942
Epoch 186 / 200 | iteration 30 / 171 | Total Loss: 2.4063804149627686 | KNN Loss: 2.396390914916992 | CLS Loss: 0.009989582002162933
Epoch 186 / 200 | iteration 40 / 171 | Total Loss: 2.385514259338379 | KNN Loss: 2.378171920776367 | CLS Loss: 0.00734240235760808
Epoch 186 / 200 | iteration 50 / 171 | Total Loss: 2.4034523963928223 | KNN Loss: 2.3900885581970215 | CLS Loss: 0.013363897800445557
Ep

Epoch 189 / 200 | iteration 50 / 171 | Total Loss: 2.4163219928741455 | KNN Loss: 2.404879331588745 | CLS Loss: 0.011442761868238449
Epoch 189 / 200 | iteration 60 / 171 | Total Loss: 2.388144016265869 | KNN Loss: 2.3862338066101074 | CLS Loss: 0.00191009440459311
Epoch 189 / 200 | iteration 70 / 171 | Total Loss: 2.4168198108673096 | KNN Loss: 2.4165663719177246 | CLS Loss: 0.0002535441890358925
Epoch 189 / 200 | iteration 80 / 171 | Total Loss: 2.3894286155700684 | KNN Loss: 2.382960081100464 | CLS Loss: 0.006468483246862888
Epoch 189 / 200 | iteration 90 / 171 | Total Loss: 2.3842203617095947 | KNN Loss: 2.3822245597839355 | CLS Loss: 0.0019959076307713985
Epoch 189 / 200 | iteration 100 / 171 | Total Loss: 2.443408966064453 | KNN Loss: 2.442505121231079 | CLS Loss: 0.0009037788840942085
Epoch 189 / 200 | iteration 110 / 171 | Total Loss: 2.414090156555176 | KNN Loss: 2.3917224407196045 | CLS Loss: 0.022367624565958977
Epoch 189 / 200 | iteration 120 / 171 | Total Loss: 2.4128868579

Epoch 192 / 200 | iteration 110 / 171 | Total Loss: 2.43387508392334 | KNN Loss: 2.4193668365478516 | CLS Loss: 0.014508312568068504
Epoch 192 / 200 | iteration 120 / 171 | Total Loss: 2.4155220985412598 | KNN Loss: 2.3944644927978516 | CLS Loss: 0.021057486534118652
Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 2.4336280822753906 | KNN Loss: 2.4298341274261475 | CLS Loss: 0.0037940246984362602
Epoch 192 / 200 | iteration 140 / 171 | Total Loss: 2.422480344772339 | KNN Loss: 2.409694194793701 | CLS Loss: 0.012786205857992172
Epoch 192 / 200 | iteration 150 / 171 | Total Loss: 2.4170117378234863 | KNN Loss: 2.4160447120666504 | CLS Loss: 0.0009671399020589888
Epoch 192 / 200 | iteration 160 / 171 | Total Loss: 2.416262626647949 | KNN Loss: 2.410883903503418 | CLS Loss: 0.005378612782806158
Epoch 192 / 200 | iteration 170 / 171 | Total Loss: 2.4066991806030273 | KNN Loss: 2.402902364730835 | CLS Loss: 0.003796837292611599
Epoch: 192, Loss: 2.4150, Train: 0.9973, Valid: 0.9873, Best

Epoch 195 / 200 | iteration 170 / 171 | Total Loss: 2.4165523052215576 | KNN Loss: 2.3889713287353516 | CLS Loss: 0.02758098393678665
Epoch: 195, Loss: 2.4115, Train: 0.9972, Valid: 0.9874, Best: 0.9879
Epoch 196 / 200 | iteration 0 / 171 | Total Loss: 2.4283392429351807 | KNN Loss: 2.4202663898468018 | CLS Loss: 0.008072968572378159
Epoch 196 / 200 | iteration 10 / 171 | Total Loss: 2.3838796615600586 | KNN Loss: 2.382077932357788 | CLS Loss: 0.0018016919493675232
Epoch 196 / 200 | iteration 20 / 171 | Total Loss: 2.477060079574585 | KNN Loss: 2.4472100734710693 | CLS Loss: 0.02985006384551525
Epoch 196 / 200 | iteration 30 / 171 | Total Loss: 2.4015281200408936 | KNN Loss: 2.390451669692993 | CLS Loss: 0.011076342314481735
Epoch 196 / 200 | iteration 40 / 171 | Total Loss: 2.4147157669067383 | KNN Loss: 2.410370111465454 | CLS Loss: 0.004345686640590429
Epoch 196 / 200 | iteration 50 / 171 | Total Loss: 2.4007561206817627 | KNN Loss: 2.390831708908081 | CLS Loss: 0.009924476966261864

Epoch 199 / 200 | iteration 50 / 171 | Total Loss: 2.4234280586242676 | KNN Loss: 2.4157543182373047 | CLS Loss: 0.007673816755414009
Epoch 199 / 200 | iteration 60 / 171 | Total Loss: 2.4474942684173584 | KNN Loss: 2.423646926879883 | CLS Loss: 0.023847365751862526
Epoch 199 / 200 | iteration 70 / 171 | Total Loss: 2.4488182067871094 | KNN Loss: 2.435717821121216 | CLS Loss: 0.013100458309054375
Epoch 199 / 200 | iteration 80 / 171 | Total Loss: 2.396406412124634 | KNN Loss: 2.3829433917999268 | CLS Loss: 0.013462989591062069
Epoch 199 / 200 | iteration 90 / 171 | Total Loss: 2.4728357791900635 | KNN Loss: 2.4690585136413574 | CLS Loss: 0.0037773221265524626
Epoch 199 / 200 | iteration 100 / 171 | Total Loss: 2.397653818130493 | KNN Loss: 2.389310598373413 | CLS Loss: 0.008343221619725227
Epoch 199 / 200 | iteration 110 / 171 | Total Loss: 2.431727409362793 | KNN Loss: 2.4261274337768555 | CLS Loss: 0.0055998992174863815
Epoch 199 / 200 | iteration 120 / 171 | Total Loss: 2.3848183155

In [8]:
test(model, test_data_iter, device)

tensor(0.9867, device='cuda:0')

In [9]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [14]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.9194646201635375


In [15]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [16]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [17]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [18]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [19]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [20]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [21]:
losses = []
accs = []
sparsity = []

In [22]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 040 | Total loss: 3.146 | Reg loss: 0.012 | Tree loss: 3.146 | Accuracy: 0.015625 | 0.885 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 01 | Batch: 000 / 040 | Total loss: 3.107 | Reg loss: 0.006 | Tree loss: 3.107 | Accuracy: 0.166016 | 0.768 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 02 | Batch: 000 / 040 

Epoch: 20 | Batch: 000 / 040 | Total loss: 2.533 | Reg loss: 0.024 | Tree loss: 2.533 | Accuracy: 0.310547 | 0.759 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 21 | Batch: 000 / 040 | Total loss: 2.529 | Reg loss: 0.024 | Tree loss: 2.529 | Accuracy: 0.314453 | 0.759 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 22 | Batch: 000 / 040 | Total loss: 2.505 | Reg loss: 0.025 | Tree loss: 2.505 | Accuracy: 0.322266 | 0.759 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 40 | Batch: 000 / 040 | Total loss: 2.398 | Reg loss: 0.028 | Tree loss: 2.398 | Accuracy: 0.347656 | 0.759 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 41 | Batch: 000 / 040 | Total loss: 2.389 | Reg loss: 0.028 | Tree loss: 2.389 | Accuracy: 0.384766 | 0.759 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 42 | Batch: 000 / 040 | Total loss: 2.342 | Reg loss: 0.028 | Tree loss: 2.342 | Accuracy: 0.375000 | 0.759 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 60 | Batch: 000 / 040 | Total loss: 2.353 | Reg loss: 0.029 | Tree loss: 2.353 | Accuracy: 0.361328 | 0.76 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 61 | Batch: 000 / 040 | Total loss: 2.365 | Reg loss: 0.029 | Tree loss: 2.365 | Accuracy: 0.347656 | 0.76 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 62 | Batch: 000 / 040 | Total loss: 2.348 | Reg loss: 0.029 | Tree loss: 2.348 | Accuracy: 0.367188 | 0.76 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0

Epoch: 80 | Batch: 000 / 040 | Total loss: 2.288 | Reg loss: 0.031 | Tree loss: 2.288 | Accuracy: 0.365234 | 0.76 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 81 | Batch: 000 / 040 | Total loss: 2.402 | Reg loss: 0.031 | Tree loss: 2.402 | Accuracy: 0.343750 | 0.76 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 82 | Batch: 000 / 040 | Total loss: 2.367 | Reg loss: 0.031 | Tree loss: 2.367 | Accuracy: 0.357422 | 0.76 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0

...

Epoch: 303 | Batch: 000 / 040 | Total loss: 2.387 | Reg loss: 0.033 | Tree loss: 2.387 | Accuracy: 0.322266 | 0.736 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 304 | Batch: 000 / 040 | Total loss: 2.338 | Reg loss: 0.033 | Tree loss: 2.338 | Accuracy: 0.365234 | 0.736 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 305 | Batch: 000 / 040 | Total loss: 2.330 | Reg loss: 0.033 | Tree loss: 2.330 | Accuracy: 0.373047 | 0.736 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 323 | Batch: 000 / 040 | Total loss: 2.333 | Reg loss: 0.033 | Tree loss: 2.333 | Accuracy: 0.347656 | 0.735 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 324 | Batch: 000 / 040 | Total loss: 2.359 | Reg loss: 0.033 | Tree loss: 2.359 | Accuracy: 0.353516 | 0.735 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 325 | Batch: 000 / 040 | Total loss: 2.302 | Reg loss: 0.033 | Tree loss: 2.302 | Accuracy: 0.402344 | 0.735 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 343 | Batch: 000 / 040 | Total loss: 2.329 | Reg loss: 0.033 | Tree loss: 2.329 | Accuracy: 0.378906 | 0.734 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 344 | Batch: 000 / 040 | Total loss: 2.323 | Reg loss: 0.033 | Tree loss: 2.323 | Accuracy: 0.390625 | 0.734 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 345 | Batch: 000 / 040 | Total loss: 2.336 | Reg loss: 0.033 | Tree loss: 2.336 | Accuracy: 0.367188 | 0.733 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 363 | Batch: 000 / 040 | Total loss: 2.355 | Reg loss: 0.033 | Tree loss: 2.355 | Accuracy: 0.347656 | 0.733 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 364 | Batch: 000 / 040 | Total loss: 2.356 | Reg loss: 0.033 | Tree loss: 2.356 | Accuracy: 0.349609 | 0.733 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 365 | Batch: 000 / 040 | Total loss: 2.362 | Reg loss: 0.033 | Tree loss: 2.362 | Accuracy: 0.337891 | 0.733 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 383 | Batch: 000 / 040 | Total loss: 2.336 | Reg loss: 0.033 | Tree loss: 2.336 | Accuracy: 0.361328 | 0.732 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 384 | Batch: 000 / 040 | Total loss: 2.362 | Reg loss: 0.033 | Tree loss: 2.362 | Accuracy: 0.369141 | 0.732 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 385 | Batch: 000 / 040 | Total loss: 2.390 | Reg loss: 0.033 | Tree loss: 2.390 | Accuracy: 0.341797 | 0.732 sec/iter
Average sparseness: 0.9840425531914894
laye

In [23]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [25]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 8.075471698113208


# Extract Rules

# Accumulate samples in the leaves

In [26]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 53


In [27]:
method = 'greedy'

In [28]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [29]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
40
============== Pattern 8 ==============
============== Pattern 9 ==============
30
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
9615
============== Pattern 17 ==============
============== Pattern 18 ==============
8687
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
=========

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
